#Natural Language Processing Course Project (CS60075)
##Entity Oriented Polarity Detection from Document

#####IIT Kharagpur, Autumn 2022-23

####Amartya Mandal (19CS10009), Sayantan Saha (19CS30041), Srijanak De (19CS30047), Aditya Tulshiram Anantwar (19CS10006), Pramit Kumar Chandra (19CS1072)


##Install necessary packages

In [ ]:
!pip install transformers

In [ ]:
!pip install torchtext==0.11.0

In [ ]:
!pip install torchmetrics

##Imports

In [ ]:
import transformers

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
import torch.nn as nn
from transformers import AdamW, get_constant_schedule_with_warmup
import torch.optim as optim

In [ ]:
from torchmetrics import F1Score, Accuracy

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


##Using apex GPU from NVIDIA

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

##Mount Drive to save files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Parse the data file

In [ ]:
contents=""
with open(r'/content/drive/MyDrive/polarity_data.js') as f:
    contents = f.read()

In [ ]:
content=[]
entities=[]

In [ ]:
id_content = [m.start() for m in re.finditer("'content'", contents)]
id_docnoId = [m.start() for m in re.finditer("'docnoId'", contents)]
id_entities = [m.start() for m in re.finditer("'entities'", contents)]
id_headline = [m.start() for m in re.finditer("'headline'", contents)]

###Extract the entity and content from json

In [ ]:
for i in range(0,len(id_entities)):
    m = id_content[i]+len("'content'")+4
    n = id_docnoId[i]-5
    content.append(contents[m:n])

In [ ]:
for i in range(0,len(id_entities)):
    m = id_entities[i]+len("'entities'")+2
    n = id_headline[i]-4
    entities.append(contents[m:n])

In [ ]:
data = {'content':content,'entities':entities}
df = pd.DataFrame(data)

In [ ]:
for i in range(0,len(df)):
    df['entities'][i] = df['entities'][i].replace('\n','')
    df['entities'][i] = df['entities'][i].replace(' ','')
    df['entities'][i] = df['entities'][i].replace("'",'"')

###Extracting Polarity

In [ ]:
entities = []
names = []
offsets = []
polarity = []
for i in range(0,len(df)):
  id_entityId = [m.start() for m in re.finditer('"entityId"', df['entities'][i])]
  id_name = [m.start() for m in re.finditer('"name"', df['entities'][i])]
  id_offsets = [m.start() for m in re.finditer('"offsets"', df['entities'][i])]
  id_polarity = [m.start() for m in re.finditer('"polarity"', df['entities'][i])]
  # print(id_name)
  lst1 = []
  lst2 = []
  lst3 = []
  lst4 = []
  for j in range(0,len(id_entityId)):
    m = id_entityId[j]+len("'entityId'")+1
    n = id_name[j]-1
    lst1.append(df['entities'][i][m:n])
    m = id_name[j]+len("'name'")+2
    n = id_offsets[j]-1
    lst2.append(df['entities'][i][m:n])
    m = id_offsets[j]+len("'offsets'")+1
    n = id_polarity[j]-1
    lst3.append(df['entities'][i][m:n])
    m = id_polarity[j]+len("'polarity'")+1
    n = id_polarity[j]+len("'polarity'")+4
    lst4.append(df['entities'][i][m:n])
  entities.append(lst1)
  names.append(lst2)
  offsets.append(lst3)
  polarity.append(lst4)

In [ ]:
df = df.assign(entity = names)
df = df.assign(polarity = polarity)

In [ ]:
df

,content,entities,entity,polarity
0,20170201\n0A21D5765ED3B8F51065D3CF7339371E\nTe...,"[{""entityId"":14123,""name"":u""Facebook"",""offsets...","[""Facebook""]",[0.5]
1,20170201\n4BBD29849EF2018A3D34932EC4AD92AB\nHe...,"[{""entityId"":10912,""name"":u""Aetna"",""offsets"":[...","[""Aetna""]",[1.0]
2,20170201\n80EA59EAB021503F49D4277EA2083662\nTh...,"[{""entityId"":4054,""name"":u""WarwickHotels&Resor...","[""WarwickHotels&Resorts""]",[1.0]
3,20161106\nFFAB0927A3C18A19E944EE9B9CBBE5E4\nGe...,"[{""entityId"":1897,""name"":u""Volkswagen"",""offset...","[""Volkswagen""]",[0.0]
4,20161106\n31009487CAE7A3C7AC245F5A758B3B8F\nGe...,"[{""entityId"":5350,""name"":u""Volkswagen"",""offset...","[""Volkswagen""]",[0.0]
...,...,...,...,...
16835,20161122\nA0534B1C9CAEFE11B7CB55BA96AE81A8\nVW...,"[{""entityId"":8089,""name"":u""Volkswagen"",""offset...","[""Volkswagen""]","[""co]"
16836,20170111\n3A15184582AE4B286ED2F9E992D7D297\nVe...,"[{""entityId"":11922,""name"":u""Yahoo"",""offsets"":[...","[""Yahoo""]","[""co]"
16837,20161202\n5CB8129BF9385F0412815C430680A066\nFa...,"[{""entityId"":9172,""name"":u""Facebook"",""offsets""...","[""Facebook""]","[""co]"
16838,20170111\n9D4787AA15641E479BFCFA554E94E8B9\nWh...,"[{""entityId"":6858,""name"":u""Yahoo"",""offsets"":[{...","[""Yahoo""]","[""co]"


###Handling data with multiple entities

Each entity is treated as a separate Data Point

In [ ]:
data = pd.DataFrame(columns=['content','entity','polarity'])
for i in range(len(df)):
  for j in range(len(df['polarity'][i])):
    d = {'content':df['content'][i], 'entity':df['entity'][i][j], 'polarity':df['polarity'][i][j]}
    data = data.append(d, ignore_index=True)

In [ ]:
for i in range(0,len(data)):
    data['polarity'][i] = data['polarity'][i].replace('"co','contradiction')

In [ ]:
data

,content,entity,polarity
0,20170201\n0A21D5765ED3B8F51065D3CF7339371E\nTe...,"""Facebook""",0.5
1,20170201\n4BBD29849EF2018A3D34932EC4AD92AB\nHe...,"""Aetna""",1.0
2,20170201\n80EA59EAB021503F49D4277EA2083662\nTh...,"""WarwickHotels&Resorts""",1.0
3,20161106\nFFAB0927A3C18A19E944EE9B9CBBE5E4\nGe...,"""Volkswagen""",0.0
4,20161106\n31009487CAE7A3C7AC245F5A758B3B8F\nGe...,"""Volkswagen""",0.0
...,...,...,...
19859,20161122\nA0534B1C9CAEFE11B7CB55BA96AE81A8\nVW...,"""Volkswagen""",contradiction
19860,20170111\n3A15184582AE4B286ED2F9E992D7D297\nVe...,"""Yahoo""",contradiction
19861,20161202\n5CB8129BF9385F0412815C430680A066\nFa...,"""Facebook""",contradiction
19862,20170111\n9D4787AA15641E479BFCFA554E94E8B9\nWh...,"""Yahoo""",contradiction


##Saving the obtained dataframe as csv

In [ ]:
data.to_csv('/content/drive/MyDrive/dataset.csv')

#BERT - NLI Model

In [ ]:
import torch
SEED = 1111
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

##Using BERT Tokenizer

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
len(tokenizer)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

30522

In [ ]:
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
print(cls_token, sep_token, pad_token, unk_token)
cls_token_idx = tokenizer.cls_token_id
sep_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
print(cls_token_idx, sep_token_idx, pad_token_idx, unk_token_idx)

[CLS] [SEP] [PAD] [UNK]
101 102 0 100


In [ ]:
max_input_length = 512

##Helper Functions for the model

In [ ]:
def tokenize_bert(sentence):
    tokens = tokenizer.tokenize(sentence) 
    return tokens
def split_and_cut(sentence):
    tokens = sentence.strip().split(" ")
    tokens = tokens[:max_input_length]
    return tokens

def trim_sentence(sent):
    try:
        sent = sent.split()
        sent = sent[:128]
        return " ".join(sent)
    except:
        return sent

In [ ]:
#Get list of 0s 
def get_sent1_token_type(sent):
    try:
        return [0]* len(sent)
    except:
        return []
#Get list of 1s
def get_sent2_token_type(sent):
    try:
        return [1]* len(sent)
    except:
        return []
#combine from lists
def combine_seq(seq):
    return " ".join(seq)
#combines from lists of int
def combine_mask(mask):
    mask = [str(m) for m in mask]
    return " ".join(mask)

##Preprocess Dataset to fit BERT - NLI Model

gold_label is the label, sentence1 is the premise and sentence2 is the Hypothesis

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')

Set Hypothesis here

In [ ]:
hypothesis = ' is represented positively here'

In [ ]:
data = pd.DataFrame(columns = ['gold_label','sentence1','sentence2'])
for i in range(len(df)):
  pol = ''
  if df['polarity'][i] == 'contradiction':
    continue
  elif df['polarity'][i] == '0.0':
    pol = 'contradiction'
  elif df['polarity'][i] == '0.3':
    pol = 'contradiction'
  elif df['polarity'][i] == '0.5':
    pol = 'neutral'
  elif df['polarity'][i] == '0.7':
    pol = 'entailment'
  elif df['polarity'][i] == '1.0':
    pol = 'entailment'
  # id = df['content'][i].find('\n')
  # print(id)
  s1 = df['content'][i][44:]
  s2 = df['entity'][i]+hypothesis
  d = {'gold_label':pol, 'sentence1':s1, 'sentence2':s2}
  data = data.append(d, ignore_index=True)

###Final Dataset

In [ ]:
data

,gold_label,sentence1,sentence2
0,neutral,Tesla is Now Officially More Than Just About C...,"""Facebook"" is represented positively here"
1,entailment,Health insurer Aetna\'s quarterly profit beats...,"""Aetna"" is represented positively here"
2,entailment,The Capital Hotel sold to American hospitality...,"""WarwickHotels&Resorts"" is represented positiv..."
3,contradiction,German prosecutors investigate Volkswagen chai...,"""Volkswagen"" is represented positively here"
4,contradiction,German prosecutors investigate Volkswagen chai...,"""Volkswagen"" is represented positively here"
...,...,...,...
19713,entailment,Yahoo to change name to Altaba\nYahoo will ado...,"""Yahoo"" is represented positively here"
19714,entailment,Ryanair to pivot growth away from the UK\nBudg...,"""Ryanair"" is represented positively here"
19715,entailment,Russian Hackers Con Advertisers out of $5 Mill...,"""WhiteOps"" is represented positively here"
19716,entailment,Russian cybercriminals make $180 million with ...,"""WhiteOps"" is represented positively here"


###Split into train-dev-test at 60-20-20

In [ ]:
df_train, df_dev, df_test = np.split(data.sample(frac=1, random_state=19), [int(.6*len(data)), int(.8*len(data))])

In [ ]:
df_test

,gold_label,sentence1,sentence2
9564,entailment,Siklu radios deployed in Houston for video sur...,"""Siklu"" is represented positively here"
4082,contradiction,Holden could gain French flavour\nGREG KABLE\n...,"""GeneralMotors"" is represented positively here"
661,contradiction,Singapore fines banks for 1MDB Fund links\nSIN...,"""GoldmanSachsGroup"" is represented positively ..."
15048,contradiction,"The Tyson Foods, Inc. (TSN) Downgraded by Vetr...","""TysonFoods"" is represented positively here"
7103,neutral,"Britain needs to hire 400,000 workers a year t...","""Arcadis"" is represented positively here"
...,...,...,...
6452,contradiction,"Pizza Hut struggling, Yum blames weak delivery...","""YumBrands"" is represented positively here"
1043,contradiction,Yahoo!'s (YHOO) Marissa Mayer to Resign from B...,"""Yahoo"" is represented positively here"
1378,contradiction,Yahoo to change name to Altaba; Marissa Mayer ...,"""Yahoo"" is represented positively here"
17141,entailment,iPhone 8 could have curved OLED screen with a ...,"""Apple"" is represented positively here"


###Tokenize, add special tokens and masks

In [ ]:
#Get neccesary columns
df_train = df_train[['gold_label','sentence1','sentence2']]
df_dev = df_dev[['gold_label','sentence1','sentence2']]
df_test = df_test[['gold_label','sentence1','sentence2']]

# #Take small dataset
# df_train = df_train[:80000]
# df_dev = df_train[:8000]
# df_test = df_train[:8000]

#Trim each sentence upto maximum length
df_train['sentence1'] = df_train['sentence1'].apply(trim_sentence)
df_train['sentence2'] = df_train['sentence2'].apply(trim_sentence)
df_dev['sentence1'] = df_dev['sentence1'].apply(trim_sentence)
df_dev['sentence2'] = df_dev['sentence2'].apply(trim_sentence)
df_test['sentence1'] = df_test['sentence1'].apply(trim_sentence)
df_test['sentence2'] = df_test['sentence2'].apply(trim_sentence)

#Add [CLS] and [SEP] tokens
df_train['sent1'] = '[CLS] ' + df_train['sentence1'] + ' [SEP] '
df_train['sent2'] = df_train['sentence2'] + ' [SEP]'
df_dev['sent1'] = '[CLS] ' + df_dev['sentence1'] + ' [SEP] '
df_dev['sent2'] = df_dev['sentence2'] + ' [SEP]'
df_test['sent1'] = '[CLS] ' + df_test['sentence1'] + ' [SEP] '
df_test['sent2'] = df_test['sentence2'] + ' [SEP]'

#Apply Bert Tokenizer for tokeinizing
df_train['sent1_t'] = df_train['sent1'].apply(tokenize_bert)
df_train['sent2_t'] = df_train['sent2'].apply(tokenize_bert)
df_dev['sent1_t'] = df_dev['sent1'].apply(tokenize_bert)
df_dev['sent2_t'] = df_dev['sent2'].apply(tokenize_bert)
df_test['sent1_t'] = df_test['sent1'].apply(tokenize_bert)
df_test['sent2_t'] = df_test['sent2'].apply(tokenize_bert)


#Get Topen type ids for both sentence
df_train['sent1_token_type'] = df_train['sent1_t'].apply(get_sent1_token_type)
df_train['sent2_token_type'] = df_train['sent2_t'].apply(get_sent2_token_type)
df_dev['sent1_token_type'] = df_dev['sent1_t'].apply(get_sent1_token_type)
df_dev['sent2_token_type'] = df_dev['sent2_t'].apply(get_sent2_token_type)
df_test['sent1_token_type'] = df_test['sent1_t'].apply(get_sent1_token_type)
df_test['sent2_token_type'] = df_test['sent2_t'].apply(get_sent2_token_type)

#Combine both sequences
df_train['sequence'] = df_train['sent1_t'] + df_train['sent2_t']
df_dev['sequence'] = df_dev['sent1_t'] + df_dev['sent2_t']
df_test['sequence'] = df_test['sent1_t'] + df_test['sent2_t']


#Get attention mask
df_train['attention_mask'] = df_train['sequence'].apply(get_sent2_token_type)
df_dev['attention_mask'] = df_dev['sequence'].apply(get_sent2_token_type)
df_test['attention_mask'] = df_test['sequence'].apply(get_sent2_token_type)

#Get combined token type ids for input
df_train['token_type'] = df_train['sent1_token_type'] + df_train['sent2_token_type']
df_dev['token_type'] = df_dev['sent1_token_type'] + df_dev['sent2_token_type']
df_test['token_type'] = df_test['sent1_token_type'] + df_test['sent2_token_type']

#Now make all these inputs as sequential data to be easily fed into torchtext Field.
df_train['sequence'] = df_train['sequence'].apply(combine_seq)
df_dev['sequence'] = df_dev['sequence'].apply(combine_seq)
df_test['sequence'] = df_test['sequence'].apply(combine_seq)
df_train['attention_mask'] = df_train['attention_mask'].apply(combine_mask)
df_dev['attention_mask'] = df_dev['attention_mask'].apply(combine_mask)
df_test['attention_mask'] = df_test['attention_mask'].apply(combine_mask)
df_train['token_type'] = df_train['token_type'].apply(combine_mask)
df_dev['token_type'] = df_dev['token_type'].apply(combine_mask)
df_test['token_type'] = df_test['token_type'].apply(combine_mask)
df_train = df_train[['gold_label', 'sequence', 'attention_mask', 'token_type']]
df_dev = df_dev[['gold_label', 'sequence', 'attention_mask', 'token_type']]
df_test = df_test[['gold_label', 'sequence', 'attention_mask', 'token_type']]
df_train = df_train.loc[df_train['gold_label'].isin(['entailment','contradiction','neutral'])]
df_dev = df_dev.loc[df_dev['gold_label'].isin(['entailment','contradiction','neutral'])]
df_test = df_test.loc[df_test['gold_label'].isin(['entailment','contradiction','neutral'])]


In [ ]:
df_train

,gold_label,sequence,attention_mask,token_type
6743,contradiction,[CLS] uber software head leaves as past harass...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
13205,entailment,[CLS] tesla brings electric cars to super ##ca...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2390,contradiction,[CLS] qu ##al ##com is sued by apple for with ...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
15471,contradiction,[CLS] samsung \ ' s thinking of splitting in t...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
17624,entailment,[CLS] apple will use drones to compete with go...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...
4731,contradiction,[CLS] bernie sanders thinks donald trump ' s c...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
16091,contradiction,[CLS] us government accuse ##s fiat chrysler o...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
8139,entailment,[CLS] mcdonald ' s sells china business in dea...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
12962,entailment,[CLS] boeing front - runner for $ 13 . 8 billi...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [ ]:
df_test

,gold_label,sequence,attention_mask,token_type
9564,entailment,[CLS] si ##k ##lu radios deployed in houston f...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4082,contradiction,[CLS] holden could gain french flavour \ ng ##...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
661,contradiction,[CLS] singapore fines banks for 1 ##md ##b fun...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
15048,contradiction,"[CLS] the tyson foods , inc . ( ts ##n ) down ...",1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
7103,neutral,"[CLS] britain needs to hire 400 , 000 workers ...",1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...
6452,contradiction,"[CLS] pizza hut struggling , yu ##m blame ##s ...",1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1043,contradiction,[CLS] yahoo ! ' s ( y ##ho ##o ) marissa mayer...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1378,contradiction,[CLS] yahoo to change name to alta ##ba ; mari...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
17141,entailment,[CLS] iphone 8 could have curved ole ##d scree...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


###Final datasets saved for future use

In [ ]:
#Save prepared data as csv file
df_train.to_csv('/content/drive/MyDrive/nlpData/train.csv', index=False)
df_dev.to_csv('/content/drive/MyDrive/nlpData/dev.csv', index=False)
df_test.to_csv('/content/drive/MyDrive/nlpData/test.csv', index=False)

##Setting up the Model

###Helper function

In [ ]:
# To convert back attention mask and token type ids to integer.
def convert_to_int(tok_ids):
    tok_ids = [int(x) for x in tok_ids]
    return tok_ids

In [ ]:
from torchtext.legacy import data

In [ ]:
#For sequence
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = split_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)
#For label
LABEL = data.LabelField()
#For Attention mask
ATTENTION = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = split_and_cut,
                  preprocessing = convert_to_int,
                  pad_token = pad_token_idx)
#For token type ids
TTYPE = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = split_and_cut,
                  preprocessing = convert_to_int,
                  pad_token = 1)

In [ ]:
fields = [('label', LABEL), ('sequence', TEXT), ('attention_mask', ATTENTION), ('token_type', TTYPE)]

###Obtain the datasets

In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = '/content/drive/MyDrive/nlpData/',
                                        train = 'train.csv',
                                        validation = 'dev.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)

In [ ]:
LABEL.build_vocab(train_data)

###Model iterator

In [ ]:
#Create iterator
BATCH_SIZE = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.sequence),
    sort_within_batch = False, 
    device = device)

###Using BERT Transformer

In [ ]:
from transformers import BertModel
bert_model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


###Model Class

In [ ]:
class BERTNLIModel(nn.Module):
    def __init__(self,

                 bert_model,

                 hidden_dim,

                 output_dim,

                ):
        super().__init__()
        self.bert = bert_model

        embedding_dim = bert_model.config.to_dict()['hidden_size']
        self.out = nn.Linear(embedding_dim, output_dim)
    def forward(self, sequence, attn_mask, token_type):
        embedded = self.bert(input_ids = sequence, attention_mask = attn_mask, token_type_ids= token_type)[1]
        output = self.out(embedded)
        return output

###Hyperparameters

In [ ]:
#defining model
HIDDEN_DIM = 512
OUTPUT_DIM = len(LABEL.vocab)
model = BERTNLIModel(bert_model,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                        ).to('cuda')

###Initialize Model on Apex GPU

In [ ]:
optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
def get_scheduler(optimizer, warmup_steps):
    scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
    return scheduler
criterion = nn.CrossEntropyLoss().to(device)
mp = True
if mp:
    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/content/apex/apex/__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
  warnings.warn(msg, DeprecatedFeatureWarning)


###Helper functions for Train, Evaluate and Obtaining Accuracy

In [ ]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True)

    correct = (max_preds.squeeze(1)==y).float()

    return correct.sum() / len(y)

In [ ]:
fp16 = True

In [ ]:
max_grad_norm = 1

def train(model, iterator, optimizer, criterion, scheduler):
    #print(iterator)
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:

        optimizer.zero_grad() # clear gradients first
        torch.cuda.empty_cache() # releases all unoccupied cached memory 
        

        sequence = batch.sequence
        attn_mask = batch.attention_mask
        token_type = batch.token_type
        #print(sequence.size(), attn_mask.size(), token_type.size())
        #print(sequence[0])
        #print(attn_mask[0])
        #print(token_type[0])
        label = batch.label
        
        predictions = model(sequence, attn_mask, token_type)
        
        #predictions = [batch_size, 3]
        #print(predictions.size())
        
        loss = criterion(predictions, label)
        
        acc = categorical_accuracy(predictions, label)
        
        if fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
        else:
            loss.backward()
        
        optimizer.step()
        scheduler.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
f1 = F1Score(num_classes=3, average = 'macro').to('cuda')
acc = Accuracy().to('cuda')

In [ ]:
def evaluate(model, iterator, criterion):
    #print(iterator)
    epoch_loss = 0
    epoch_acc = 0
    score = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            #print(batch)

            sequence = batch.sequence
            attn_mask = batch.attention_mask
            token_type = batch.token_type
            labels = batch.label
                        
            predictions = model(sequence, attn_mask, token_type)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            score += 1e5*f1(predictions,labels).item()
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # print(score)
            # print(epoch_acc)
        # print(classification_report(pred,lab))
    return epoch_loss / len(iterator), epoch_acc / len(iterator), score / len(iterator)

###Helper Function to compute time taken

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
len(train_data)

11830

##Train the Model

In [ ]:
import math
N_EPOCHS = 6

warmup_percent = 0.2
total_steps = math.ceil(N_EPOCHS*len(train_data)*1./BATCH_SIZE)
warmup_steps = int(total_steps*warmup_percent)
scheduler = get_scheduler(optimizer, warmup_steps)

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, scheduler)
    valid_loss, valid_acc, _ = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/nlpData/bert-nli.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Epoch: 01 | Epoch Time: 5m 37s
	Train Loss: 0.160 | Train Acc: 95.43%
	 Val. Loss: 0.175 |  Val. Acc: 95.42%
Epoch: 02 | Epoch Time: 5m 38s
	Train Loss: 0.122 | Train Acc: 96.85%
	 Val. Loss: 0.161 |  Val. Acc: 96.41%
Epoch: 03 | Epoch Time: 5m 34s
	Train Loss: 0.078 | Train Acc: 98.01%
	 Val. Loss: 0.151 |  Val. Acc: 96.69%
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Epoch: 04 | Epoch Time: 5m 34s
	Train Loss: 0.049 | Train Acc: 98.78%
	 Val. Loss: 0.150 |  Val. Acc: 96.81%
Epoch: 05 | Epoch Time: 5m 34s
	Train Loss: 0.036 | Train Acc: 99.09%
	 Val. Loss: 0.189 |  Val. Acc: 96.79%
Epoch: 06 | Epoch Time: 5m 33s
	Train Loss: 0.027 | Train Acc: 99.27%
	 Val. Loss: 0.163 |  Val. Acc: 96.96%


##Test the Model

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/nlpData/bert-nli.pt'))
test_loss, test_acc, f1_score = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}% |  f1 score: {f1_score*1e-5:.6f}')

Test Loss: 0.176 |  Test Acc: 96.46% |  f1 score: 0.925843


In [ ]:
def predict_inference(premise, hypothesis, model, device):
    
    model.eval()
    
    premise = '[CLS] ' + premise + ' [SEP]'
    hypothesis = hypothesis + ' [SEP]'
    
    prem_t = tokenize_bert(premise)
    hypo_t = tokenize_bert(hypothesis)
    
    #print(len(prem_t), len(hypo_t))
    
    prem_type = get_sent1_token_type(prem_t)
    hypo_type = get_sent2_token_type(hypo_t)
    
    #print(len(prem_type), len(hypo_type))
    
    indexes = prem_t + hypo_t
    
    indexes = tokenizer.convert_tokens_to_ids(indexes)
    #print(indexes)
    indexes_type = prem_type + hypo_type
    #print(indexes_type)
    
    attn_mask = get_sent2_token_type(indexes)
    #print(attn_mask)
    
    #print(len(indexes))
    #print(len(indexes_type))
    #print(len(attn_mask))

    #seq = '[CLS] '+ premise + ' [SEP] '+ hypothesis 

    #tokens = tokenizer.tokenize(seq)

    #indexes = tokenizer.convert_tokens_to_ids(tokens)
    
    indexes = torch.LongTensor(indexes).unsqueeze(0).to(device)
    indexes_type = torch.LongTensor(indexes_type).unsqueeze(0).to(device)
    attn_mask = torch.LongTensor(attn_mask).unsqueeze(0).to(device)
    
    #print(indexes.size())
    
    prediction = model(indexes, attn_mask, indexes_type)
    
    prediction = prediction.argmax(dim=-1).item()
    
    return LABEL.vocab.itos[prediction]

In [ ]:
def get_prediction(content, entity, model, device):
  hypothesis = entity + ' is represented positively here'
  label = predict_inference(content, hypothesis, model, device)
  if label == 'neutral':
    return 'neutral'
  elif label == 'entailment':
    return 'positive'
  return 'negative'

In [ ]:
content = 'Recently, huge criticisms have rocked Facebook.'
entity = 'Facebook'

get_prediction(content, entity, model, device)

'negative'